## chroma db 빌드

In [2]:
!pip install jupyterlab_widgets ipywidgets


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
import os
from tqdm.notebook import tqdm

os.makedirs("chroma_db", exist_ok=True)
# 데이터 불러오기
df = pd.read_csv("unipass_crawling_1005.csv")
df = df.dropna(subset=["이름", "설명", "사유"]) # NaN 제거

# 임베딩 모델 불러오기(paraphrase-multilingual-MiniLM-L12-v2)
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Chroma 클라이언트 생성 (로컬 DB)
client = chromadb.PersistentClient(path="chroma_db")
collection = client.get_or_create_collection("hscode_collection")

### 각 row 임베딩해서 저장 ###
# embedding: embedding 결과            ex. [0.1, 0.2, 0.3]
# documents: 원본 데이터               ex. "상품명: 베어링용 고무 씰 \n 설명: 철강제 링과 가황환 고무제 링이 결합된 Wheel Hub Bearing 측면 밀봉용 Seal."
# metadata: HSCode, 상품명, 시행일자   ex. {"HSCode": "1234567890", "상품명": "베어링용 고무 씰", "시행일자": "2025-01-01"}
for i, row in tqdm(df.iterrows(), total=len(df), desc="Embedding & Inserting"):
    text = f"상품명: {row['이름']}\n설명: {row['설명']}\n사유: {row['사유']}"
    emb = model.encode(text).tolist()

    collection.add(
        embeddings=[emb],
        documents=[text],
        metadatas=[{
            "HSCode": row["HSCode"],
            "상품명": row["이름"],
            "시행일자": row["시행일자"]
        }],
        ids=[str(i)]
    )
client.persist()

Embedding & Inserting:   0%|          | 0/29943 [00:00<?, ?it/s]

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# 사용자 쿼리
query = "상품명: 베어링용 고무 씰 \n 설명: 철강제 링과 가황환 고무제 링이 결합된 Wheel Hub Bearing 측면 밀봉용 Seal."

# 쿼리 임베딩
query_emb = model.encode([query]).tolist()


# query = "휴대용 전기밥솥, 소형 전기 조리기기"
# query_emb = model.encode(query, normalize_embeddings=True).tolist()

# DB 불러오기
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection("hscode_collection")

# 쿼리 기반 DB 검색 결과
results = collection.query(
    query_embeddings=query_emb,
    n_results=3
)


# result 출력
for metadata, doc, dist in zip(results["metadatas"][0], results["documents"][0], results["distances"][0]):
    print("상품명:", metadata.get("상품명"))
    print("HSCode:", metadata.get("HSCode"))
    print("시행일자:", metadata.get("시행일자"))
    print("유사도 거리:", round(dist, 3))
    print("-"*50)


상품명: Rubber Seal; 300007957-0000 / F-679631.05-0061.DH.RDL;
HSCode: 8487.90-9010
시행일자: 2025-08-14
유사도 거리: 4.41
--------------------------------------------------
상품명: Other Machinery parts; Rubber Seal; 097014290-0000 / F-679336-0161.DSH.RDL;
HSCode: 8487.90-9010
시행일자: 2025-08-14
유사도 거리: 5.84
--------------------------------------------------
상품명: Rubber Seal; 066839483-0000 / F-573620-0061.DC.RDL;
HSCode: 8487.90-9010
시행일자: 2025-08-18
유사도 거리: 6.135
--------------------------------------------------


In [ ]:
print("저장된 데이터 개수:", collection.count())

sample = collection.get(limit=3)  # 예시 데이터
print(sample)

저장된 데이터 개수: 12380
{'ids': ['0', '1', '2'], 'embeddings': None, 'documents': ['상품명: Rubber Seal; 066839483-0000 / F-573620-0061.DC.RDL;\n설명: - 개요 : 철강제 링과 가황환 고무제 링이 결합된 Wheel Hub Bearing 측면 밀봉용 Seal\n\n- 용도 : 베어링 내부의 그리스가 외부로 누설되는 것을 막고, 외부 이물질 침입 차단 기능 수행\n\n- 크기 : 외경 약 69 mm, 두께 약 4mm\n사유: - 관세율표 제16부 주 제2호 다목에서 “그 밖의 각종 부분품은 경우에 따라 제8409호ㆍ제8431호ㆍ제8448호ㆍ제8466호ㆍ제8473호ㆍ제8503호ㆍ제8522호ㆍ제8529호ㆍ제8538호로 분류하거나 위의 호로 분류하지 못하는 경우에는 제8487호나 제8548호로 분류한다.”라고 규정함\n\n- 관세율표 제8487호에는 “기계류의 부분품(접속자ㆍ절연체ㆍ코일ㆍ접촉자와 그 밖의 전기용품을 포함하지 않으며, 이 류에 따로 분류되지 않은 것으로 한정한다)”이 분류되고,\n․같은 호 해설서에서 “..(전략).. 이 호에 속하는 물품은 일반적으로 기계의 부분품으로서 인정될 수 있는 것이지만, 특정 기계의 부분품은 제외한다. 이러한 조건에 따라 이 호에는 비자동식의 윤활유용 포트 ; 그리스용의 니플(nipple) ; 수동식 휠․레버와 손잡이 ; 안전덮개․베이스플레이트(baseplate) ; 오일 실 링(oil seal ring)을 포함한다. 일반적으로 횡단면이 원형인 이들 링(ring)은 움직이는 부분이 없는 것이 특징이며 구조가 대단히 간단하다[예를 들면, 플렉시블한(flexible) 고무링과 금속보강재를 경화(vulcanisation)하여 조립한 것]. 이들 물품은 수많은 기계와 기기에서 연결 표면을 밀폐하여 기름이나 가스의 누출을 방지하거나 먼지 등이 들어오는 것을 방지하는데 사용한다.”고 설명하고 있음\n\n- 따라서, 본 물품은 베어링 내부의 그

## hscode data 임베딩

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings


# CSV 파일 경로 설정
file_path = r'C:\Users\user\OneDrive\Desktop\hs_code\embedding\hscode_data.csv'

# HS Code 매핑 데이터 불러오기
df_hs = pd.read_csv(file_path)
df_hs.dropna(inplace=True)
# 모델 로드
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Chroma 클라이언트 생성 (기존 DB 경로 지정)
client = chromadb.PersistentClient(path='/chroma_db')

# 기존 컬렉션 불러오기 (없으면 생성)
collection = client.get_or_create_collection("hscode_collection")


for i, row in df_hs.iterrows():
    # 한글 품목명이 없으면 영문만 사용
    korean_name = row["한글품목명"] if pd.notna(row["한글품목명"]) else ""
    english_name = row["영문품목명"] if pd.notna(row["영문품목명"]) else ""

    # 검색용 텍스트(document)
    text = (
        f"HS 코드: {row['HS부호']}\n"
        f"한글 품목명: {korean_name}\n"
        f"영문 품목명: {english_name}\n"
        f"parent_code: {row['parent_code']}"
    )

    emb = model.encode(text).tolist()

    # 메타데이터
    metadata = {
        "HSCode": str(row["HS부호"]),
        "한글품목명": korean_name,
        "영문품목명": english_name,
        "parent_code": str(row["parent_code"]),
        "category": "mapping"
    }

    if "한국표준무역분류명" in row and pd.notna(row["한국표준무역분류명"]):
        metadata["한국표준무역분류명"] = row["한국표준무역분류명"]
    if "품목규격명" in row and pd.notna(row["품목규격명"]):
        metadata["품목규격명"] = row["품목규격명"]

    collection.add(
        embeddings=[emb],
        documents=[text],
        metadatas=[metadata],
        ids=[f"hsmap_{i}"]
    )


c:\Users\user\anaconda3\envs\mw\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
